## Step 9: Store the Model

Save the trained model to the Object Storage system configured in your Workbench connection. 

Start by getting the credentials and configuring variables for accessing Object Storage.

In [ ]:
# Imports
import os
import boto3
from botocore.exceptions import NoCredentialsError, PartialCredentialsError

Deserialize from file.

In [ ]:
import torch 

results_train_save_path = "model_train_results.pth"
results_train = torch.load(results_train_save_path)

results_test_save_path = "model_test_results.pth"
results_test = torch.load(results_test_save_path)


In [ ]:
AWS_S3_ENDPOINT = os.getenv("S3_ENDPOINT", "").replace('https://', '').replace('http://', '')
AWS_ACCESS_KEY_ID = os.getenv("ACCESS_KEY")
AWS_SECRET_ACCESS_KEY = os.getenv("SECRET__KEY")
AWS_S3_BUCKET = os.getenv("S3_BUCKET")

Define the S3 client.

In [ ]:
s3_client = boto3.client(
    "s3",
    endpoint_url=f"https://{AWS_S3_ENDPOINT}",  
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    verify=True
)

Select files to be uploaded (files generated while training and validating the model)

In [ ]:
model_path_train = results_train['save_dir']
weights_path = os.path.join(model_path_train, "weights")
model_path_test = results_test['save_dir']

# Get file lists
files_train = [os.path.join(model_path_train, f) for f in os.listdir(model_path_train) if os.path.isfile(os.path.join(model_path_train, f))]
files_models = [os.path.join(weights_path, f) for f in os.listdir(weights_path) if os.path.isfile(os.path.join(weights_path, f))]
files_test = [os.path.join(model_path_test, f) for f in os.listdir(model_path_test) if os.path.isfile(os.path.join(model_path_test, f))]

directory_name = os.path.basename(model_path_train)

Upload the files.

In [ ]:
def upload_file(file_path, s3_path):
    try:
        s3_client.upload_file(file_path, AWS_S3_BUCKET, s3_path)
        print(f"'{os.path.basename(file_path)}' uploaded successfully to '{s3_path}'.")
    except (NoCredentialsError, PartialCredentialsError) as e:
        print("Credentials error: ", e)
    except Exception as e:
        print("Error occurred: ", e)

# Upload train files
for file_path in files_train:
    upload_file(file_path, f"prototype/pipeline/{directory_name}/train-val/{os.path.basename(file_path)}")

# Upload model weights
for file_path in files_models:
    upload_file(file_path, f"prototype/pipeline/{directory_name}/{os.path.basename(file_path)}")

# Upload test files
for file_path in files_test:
    upload_file(file_path, f"prototype/pipeline/{directory_name}/test/{os.path.basename(file_path)}")

## Step 10: Remove local files

Once you uploaded the Model data to the Object Storage, you can remove the local files to save disk space and also the files where we stored the variable values.

In [ ]:
!rm -rf {model_path_train}
!rm -rf {model_path_test}
!rm -rf {results_train_save_path}
!rm -rf {results_test_save_path}